# PostgreSQL

## SELECT Query

```sql
SELECT DISTINCT column1, column2, NOW() AS current_time
FROM table_name
WHERE condition1
    AND condition2
    OR condition3
ORDER BY column1 ASC, column2 DESC NULLS LAST -- can also be NULLS FIRST
OFFSET 10 LIMIT 10;
```

## Operators

| Operator | Description |
| --- | --- |
| `=`, `<>`, `!=`, `>`, `<`, `>=`, `<=` | Equal, Not Equal, Greater Than, Less Than, Greater Than or Equal, Less Than or Equal |
| `::` | Cast a value to a different data type |
| `BETWEEN` | Between a range, like `payment_date BETWEEN '2007-02-15' AND '2007-02-20'` |
| `IN` | In a list of values, like `col_name IN ('val1', 'val2')` |
| `LIKE` | Pattern matching, `%` for any number of characters, `_` for a single character |
| `IS NULL`, `IS NOT NULL` | Check if value is NULL or not NULL |
| `AND`, `OR`, `NOT` | Logical operators |
| `ANY`, `ALL` | Compare a value to a set of values, like `col_name > ANY (1, 2, 3)` |
| `EXISTS`, `NOT EXISTS` | Check if a subquery returns any rows |

## JOINs

`INNER JOIN`, `LEFT JOIN` = `LEFT OUTER JOIN`, `RIGHT JOIN` = `RIGHT OUTER JOIN`, `FULL JOIN` = `FULL OUTER JOIN`, `CROSS JOIN`, `NATURAL JOIN` are supported.

## UNION, UNION ALL, INTERSECT, EXCEPT

`UNION` removes duplicates, `UNION ALL` keeps duplicates, `INTERSECT` returns common rows, `EXCEPT` returns rows in first query but not in second query.

## Boolean Operators

- Boolean can have three values: `true`, `false`, and `null`.
- `true` -> true, `t`, `y`, `yes`, `true`, `1`
- `false` -> false, `f`, `n`, `no`, `false`, `0`

## GROUP BY (grouping sets)

```sql
SELECT
	GROUPING(brand) grouping_brand, -- 1 if brand is NULL, 0 otherwise
	GROUPING(segment) grouping_segment,
	brand,
	segment,
	SUM (quantity)
FROM
	sales
GROUP BY
	GROUPING SETS ( -- multiple GROUP BYs, 
        (brand, segment), -- group by brand and segment
		(brand), -- group by brand, segment is NULL
		(segment), -- group by segment, brand is NULL
		() -- no grouping
	)
HAVING GROUPING(brand) = 0 -- only show rows where brand is not NULL
ORDER BY
	brand,
	segment;
```

## GROUP BY (full rollup)

```sql
SELECT segment, brand, SUM (quantity) FROM sales 
GROUP BY 
    ROLLUP (segment, brand) -- full rollup, generates all levels in hierarchy
ORDER BY segment, brand
-- above is equivalent to
SELECT segment, brand, SUM (quantity) FROM sales 
GROUP BY 
    GROUPING SETS (
      (segment, brand),
      (segment),
      ()
    )
ORDER BY segment, brand
```

## GROUP BY (partial rollup)

```sql
SELECT segment, brand, SUM (quantity) FROM sales 
GROUP BY 
    segment, ROLLUP (brand) -- partial rollup
ORDER BY segment, brand
-- above is equivalent to
SELECT segment, brand, SUM (quantity) FROM sales 
GROUP BY 
    GROUPING SETS (
      (segment, brand),
      (segment)
    )
ORDER BY segment, brand
```

## GROUP BY (cube)

```sql
SELECT segment, brand, SUM (quantity) FROM sales
GROUP BY
    CUBE (brand, segment) -- generate all possible combinations
ORDER BY segment, brand
-- above is equivalent to
SELECT segment, brand, SUM (quantity) FROM sales
GROUP BY
    GROUPING SETS (
      (segment, brand),
      (segment),
      (brand),
      ()
    )
ORDER BY segment, brand
```

## ANY and ALL

```sql
SELECT - FROM employees
WHERE salary < ALL (SELECT salary FROM managers)
-- use ANY / ALL with <, >, =, <=, >=, <> and subquery
ORDER BY salary DESC;
```

## EXISTS

```sql
SELECT first_name, last_name FROM customer c
WHERE EXISTS ( -- check if subquery returns any rows
    SELECT 1 FROM payment p
    WHERE p.customer_id = c.customer_id AND amount > 11 -- correlated subquery
)
```

## Common Table Expressions (CTE)

```sql
WITH film_stats AS (
    SELECT AVG(rental_rate) AS avg_rental_rate, MAX(length) AS max_length, MIN(length) AS min_length
    FROM film
), customer_stats AS (
    SELECT COUNT(DISTINCT customer_id) AS total_customers, SUM(amount) AS total_payments
    FROM payment
), main_query AS (
    SELECT
        ROUND((SELECT avg_rental_rate FROM film_stats), 2) AS avg_film_rental_rate,
        (SELECT max_length FROM film_stats) AS max_film_length,
        (SELECT min_length FROM film_stats) AS min_film_length,
        (SELECT total_customers FROM customer_stats) AS total_customers,
        (SELECT total_payments FROM customer_stats) AS total_payments
) SELECT - FROM main_query;
```

## Recursive CTE

- use `WITH RECURSIVE` to create recursive CTE
```sql
WITH RECURSIVE subordinates AS (
  SELECT employee_id, manager_id, full_name FROM employees
  WHERE employee_id = 2
  UNION
  SELECT e.employee_id, e.manager_id, e.full_name FROM employees e
    INNER JOIN subordinates s ON s.employee_id = e.manager_id
)
SELECT - FROM subordinates;
```

## Modifying Data, inserting rows

```sql
CREATE TABLE contacts (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    email VARCHAR(384) NOT NULL UNIQUE
);
```

```sql
INSERT INTO contacts (first_name, last_name, email)
VALUES
    ('John', 'Doe', '[[email protected]](../cdn-cgi/l/email-protection1.html)'),
    ('Jane', 'Smith', '[[email protected]](../cdn-cgi/l/email-protection2.html)'),
    ('Bob', 'Johnson', '[[email protected]](../cdn-cgi/l/email-protection3.html)')
RETURNING *;
```

## Modifying Data, updating rows

```sql
CREATE TABLE courses(
  course_id serial PRIMARY KEY,
  course_name VARCHAR(255) NOT NULL,
  price DECIMAL(10,2) NOT NULL,
  description VARCHAR(500),
  published_date date
);

INSERT INTO courses( course_name, price, description, published_date)
VALUES
('PostgreSQL for Developers', 299.99, 'A complete PostgreSQL for Developers', '2020-07-13'),
('PostgreSQL Admininstration', 349.99, 'A PostgreSQL Guide for DBA', NULL),
('PostgreSQL High Performance', 549.99, NULL, NULL),
('PostgreSQL Bootcamp', 777.99, 'Learn PostgreSQL via Bootcamp', '2013-07-11'),
('Mastering PostgreSQL', 999.98, 'Mastering PostgreSQL in 21 Days', '2012-06-30');

UPDATE courses
SET published_date = '2020-08-01'
WHERE course_id = 3;

UPDATE courses
SET price = price - 1.05;
```

## Modifying Data, upserting rows

```sql
INSERT INTO inventory (id, name, price, quantity)
VALUES (1, 'A', 16.99, 120)
ON CONFLICT(id)
DO UPDATE SET
  price = EXCLUDED.price,
  quantity = EXCLUDED.quantity;
```

## Transactions

- Operations that maintain data integrity using BEGIN, COMMIT, and ROLLBACK statements to ensure all-or-nothing execution
```sql
BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id = 1;
UPDATE accounts SET balance = balance + 100 WHERE id = 2;
COMMIT;  -- or ROLLBACK if any errors occur
```

## Data Import/Export
- Use COPY command to move data between CSV files and PostgreSQL tables
```sql
COPY employees(name, salary, department)
FROM '/path/to/employees.csv' 
DELIMITER ',' 
CSV HEADER;
```

## Table Management
- Create, modify, and manage database tables using DDL commands
```sql
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    price NUMERIC(10,2),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

## Constraints
- Rules enforced on columns to maintain data integrity
```sql
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    user_id INT REFERENCES users(id) ON DELETE CASCADE,
    total_amount NUMERIC(10,2) CHECK (total_amount > 0),
    email VARCHAR(255) UNIQUE NOT NULL
);
```

## Data Types
- Specialized types for storing different kinds of data efficiently
```sql
CREATE TABLE events (
    id SERIAL PRIMARY KEY,
    title VARCHAR(200),
    description TEXT,
    is_active BOOLEAN DEFAULT true,
    tags TEXT[],  -- Array type
    metadata JSONB,  -- JSON type
    start_time TIMESTAMP WITH TIME ZONE,
    duration INTERVAL
);
```

## Conditional Expressions
- Logic control using CASE, COALESCE, and NULLIF for data manipulation
```sql
SELECT 
    name,
    CASE 
        WHEN salary > 100000 THEN 'High'
        WHEN salary > 50000 THEN 'Medium'
        ELSE 'Low'
    END as salary_bracket,
    COALESCE(department, 'Unassigned') as department
FROM employees;
```